Math 5750/6880: Mathematics of Data Science \
Project 3

# 1. Fashion-MNIST image classification using sklearn

In [2]:
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape(len(X_train), -1)
X_test  = X_test.reshape(len(X_test), -1)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
def run_configuration(config):
    mlp = MLPClassifier(**config)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    test_item = {"config":config, "accuracy":accuracy_score(y_test, y_pred)}
    return test_item

In [5]:
import random
import math
import warnings
from sklearn.exceptions import ConvergenceWarning
global population

config_ranges = {
    "hidden_layer_sizes": [(100,), (50), (10, 10)],
    "max_iter": [7],
    "alpha": [1e-3, 1e-4, 1e-5],
    "solver": ["adam", "sgd"],
    "learning_rate_init": [0.05, 0.1, 0.15],
    "activation": ['relu', 'logistic', 'tanh'],
    "verbose": [0],
    "random_state": [1]
}
numerical_keys = {"alpha", "learning_rate_init"}
all_keys = list(config_ranges.keys())
mutable_keys = [key for key in all_keys if len(config_ranges[key]) > 1]

mutant_rate = 0.25
max_population = 10
generations = 7
births_per_generation = 5
tournament_size = 4
population = []

def sort_population():
    global population
    population = sorted(population, key=lambda item: -item["accuracy"])

def add_to_population(config):
    result = run_configuration(config)
    if result not in population:
       population.insert(1, result)

def random_tournament(ignore={}):
    volunteers = [body for body in population if body != ignore]
    return max(random.sample(volunteers, min(len(volunteers), tournament_size)), key=lambda body: body["accuracy"])

def reproduce(left_body, right_body):
    new_config = left_body["config"].copy()
    for each_key in random.sample(all_keys, random.randint(0, len(all_keys))):
        new_config[each_key] = right_body["config"][each_key]
    if random.uniform(0, 1) < mutant_rate:
        mutation = random.choice(mutable_keys)
        if mutation in numerical_keys:
            new_config[mutation] *= random.uniform(0.8, 1.2)
        else:
            new_config[mutation] = random.choice([each for each in config_ranges[mutation] if each != new_config[mutation]])

    return new_config

def cull_population():
    sort_population()
    global population
    population = population[:max_population]

def init_population():
    for _ in range(0, max_population):
        new_config = {}
        for each_key in config_ranges:
            new_config[each_key] = random.choice(config_ranges[each_key])
        add_to_population(new_config)

# Filter out ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

init_population()

for generation in range(0, generations + 1):
    print("generation", generation, "population", len(population), "best", max(population, key=lambda body: body["accuracy"])["accuracy"])
    for _ in range(0, births_per_generation):
        left_body = random_tournament()
        right_body = random_tournament(left_body)
        add_to_population(reproduce(left_body, right_body))
    cull_population()

generation 0 population 10 best 0.8772
generation 1 population 10 best 0.8773
generation 2 population 10 best 0.8773
generation 3 population 10 best 0.8775
generation 4 population 10 best 0.8794
generation 5 population 10 best 0.8794
generation 6 population 10 best 0.8794
generation 7 population 10 best 0.8798


In [6]:

for each_body in population: print(each_body)



{'config': {'hidden_layer_sizes': (100,), 'max_iter': 7, 'alpha': 0.0010062195987208915, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8798}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 7, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8794}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 7, 'alpha': 0.0010062195987208915, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'tanh', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8775}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 7, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'tanh', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8773}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 7, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.056379950688487716, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'a

# 3. Fashion-MNIST image classification  using pytorch

In [9]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# scale to [0,1], add channel dimension -> (N, 1, 28, 28)
X_train = (X_train.astype("float32") / 255.0)[:, None, :, :]
X_test  = (X_test.astype("float32")  / 255.0)[:,  None, :, :]

y_train = y_train.astype(np.int64)
y_test  = y_test.astype(np.int64)

# train/val split: last 10k of train as validation
X_tr, X_val = X_train[:50000], X_train[50000:]
y_tr, y_val = y_train[:50000], y_train[50000:]

# wrap in PyTorch TensorDatasets and DataLoaders
train_ds = TensorDataset(torch.from_numpy(X_tr),  torch.from_numpy(y_tr))
val_ds   = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_ds  = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

# 2. Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
import torch.nn as nn
import torch.optim as optim

# In colab, you should ``change runtime type'' to GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# your code here

Using device: cuda


In [10]:
# 1. Define a Convolutional Neural Network (CNN) model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 7 * 7, 10) # 7x7 comes from the image size after pooling

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1) # Flatten the output for the fully connected layer
        x = self.fc(x)
        return x

model = SimpleCNN().to(device)

In [25]:
# 6. Evaluate the trained model on the test set
model.eval() # Set the model to evaluation mode
with torch.no_grad(): # Disable gradient calculation for evaluation
    correct = 0
    total = 0
    for images, labels in test_loader: # Evaluate on the test set
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 90.39%


In [23]:
# 4. Implement the evaluation loop
model.eval() # Set the model to evaluation mode
with torch.no_grad(): # Disable gradient calculation for evaluation
    correct = 0
    total = 0
    for images, labels in val_loader: # Evaluate on the validation set
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

Validation Accuracy: 91.40%


In [21]:
# 3. Implement the training loop
num_epochs = 10 # You can adjust this number

for epoch in range(num_epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

Epoch [1/10], Loss: 0.1089
Epoch [2/10], Loss: 0.1094
Epoch [3/10], Loss: 0.1058
Epoch [4/10], Loss: 0.1060
Epoch [5/10], Loss: 0.1040
Epoch [6/10], Loss: 0.1007
Epoch [7/10], Loss: 0.0999
Epoch [8/10], Loss: 0.0995
Epoch [9/10], Loss: 0.0990
Epoch [10/10], Loss: 0.0968
